In [1]:
import xml.etree.ElementTree as ET
from os import path,makedirs
import requests

### Adding directory maniluplation to repsect format

In [3]:
%%bash
mkdir -p /nrcan_p2/data/01_raw
mkdir -p /nrcan_p2/data/02_intermediate
mkdir -p /nrcan_p2/data/03_primary
mkdir -p /nrcan_p2/data/04_feature
mkdir -p /nrcan_p2/data/05_model_input
mkdir -p /nrcan_p2/data/06_models
mkdir -p /nrcan_p2/data/07_model_output
mkdir -p /nrcan_p2/data/08_reporting

In [2]:
root_dir = "/nrcan_p2"
data_dir = path.join(root_dir,"data")
geoscan_files_dir=join(data_dir,"01_raw","20201006","geoscan")
raw_dir=path.join(geoscan_files_dir,"raw")

In [3]:
if not path.exists(raw_dir):
    os.mmakedirs(raw_dir)

In [4]:
def get_doi_geoid_geourl(xml_item):
    doi=None
    geo_id=None
    geo_url=None
    identifiers = xml_item.findall("{http://purl.org/dc/elements/1.1/}identifier")
    for identifier in identifiers:
        text = identifier.text
        if "info:doi" in text:
            doi = text
        elif "geoscanid:" in text:
            geo_id = text.split(":")[1]
        elif "ftp.geogratis.gc.ca" in text:
            geo_url = text
    return doi,geo_id,geo_url

In [5]:
tree = ET.parse(path.join(geoscan_files_dir,"EAIDown.xml"))
root= tree.getroot()

items_to_download = []
for item in root:
    items_to_download.append((item,*get_doi_geoid_geourl(item)))

In [17]:
print(f'Number of files to download: {len(items_to_download)}')

Number of files to download: 12316


In [6]:
no_doi = []
no_geo_id = []
no_geo_url = []
missing_data = "doi,geoscan_id,download_url\n"

for item in items_to_download:
    if not all(item):
        if item[1] is None:
            no_doi.append(item)
        if item[2] is None:
            no_geo_id.append(item)
        if item[3] is None:
            no_geo_url.append(item)
        missing_data+=f'{item[1]},{item[2]},{item[3]}\n'  

In [7]:
print(f'Missing doi:{len(no_doi)}, Missing geoscan id: {len(no_geo_id)}, Missing metadata url:{len(no_geo_url)} ')

Missing doi:29, Missing geoscan id: 0, Missing metadata url:248 


In [8]:
with open(path.join(geoscan_files_dir,"missing_id_or_url.csv"),"w") as f:
    f.write(missing_data)

In [9]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
def download_file(url,save_path):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True,verify=False) as r:
        r.raise_for_status()
        with open(save_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

In [10]:
import time

failed_to_download = []
count =0
for item in items_to_download:
    geo_id = item[2]
    geo_url = item[3]
    if (geo_id is None) or (geo_url is None):
        # do nothing since we can no dowload and rename the file
        pass
    else:
        try:
            zip_path = path.join(raw_dir,geo_id+"."+geo_url.split(".")[-1])
            if path.exists(zip_path):
                print(f'{zip_path} already exists')
            else:
                print(f'Downloading: {zip_path}')
                start_time = time.time()
                download_file(geo_url,zip_path)
                #resp = requests.get(geo_url, verify=False)
                #with open(zip_path,"wb") as f:
                #    f.write(resp.content)
                print(f'Succeeded: {zip_path} in {(time.time() - start_time):.2f}')

        except Exception as e:
            failed_to_download.append((item,e))

/nrcan_p2/data/geoscan/raw/407.zip already exists
/nrcan_p2/data/geoscan/raw/4680.pdf already exists
/nrcan_p2/data/geoscan/raw/4681.pdf already exists
/nrcan_p2/data/geoscan/raw/4682.pdf already exists
/nrcan_p2/data/geoscan/raw/8273.pdf already exists
/nrcan_p2/data/geoscan/raw/8274.pdf already exists
/nrcan_p2/data/geoscan/raw/8275.pdf already exists
/nrcan_p2/data/geoscan/raw/8276.pdf already exists
/nrcan_p2/data/geoscan/raw/8277.pdf already exists
/nrcan_p2/data/geoscan/raw/8278.pdf already exists
/nrcan_p2/data/geoscan/raw/8279.pdf already exists
/nrcan_p2/data/geoscan/raw/8280.pdf already exists
/nrcan_p2/data/geoscan/raw/8281.pdf already exists
/nrcan_p2/data/geoscan/raw/8282.pdf already exists
/nrcan_p2/data/geoscan/raw/8283.pdf already exists
/nrcan_p2/data/geoscan/raw/8284.pdf already exists
/nrcan_p2/data/geoscan/raw/8285.pdf already exists
/nrcan_p2/data/geoscan/raw/8286.pdf already exists
/nrcan_p2/data/geoscan/raw/8287.pdf already exists
/nrcan_p2/data/geoscan/raw/8288.

/nrcan_p2/data/geoscan/raw/8580.pdf already exists
/nrcan_p2/data/geoscan/raw/8581.pdf already exists
/nrcan_p2/data/geoscan/raw/8582.pdf already exists
/nrcan_p2/data/geoscan/raw/8583.zip already exists
/nrcan_p2/data/geoscan/raw/8584.pdf already exists
/nrcan_p2/data/geoscan/raw/8588.pdf already exists
/nrcan_p2/data/geoscan/raw/8589.pdf already exists
/nrcan_p2/data/geoscan/raw/8590.pdf already exists
/nrcan_p2/data/geoscan/raw/8591.pdf already exists
/nrcan_p2/data/geoscan/raw/8592.pdf already exists
/nrcan_p2/data/geoscan/raw/8593.pdf already exists
/nrcan_p2/data/geoscan/raw/8594.pdf already exists
/nrcan_p2/data/geoscan/raw/8595.pdf already exists
/nrcan_p2/data/geoscan/raw/8596.zip already exists
/nrcan_p2/data/geoscan/raw/8597.pdf already exists
/nrcan_p2/data/geoscan/raw/8600.pdf already exists
/nrcan_p2/data/geoscan/raw/8602.pdf already exists
/nrcan_p2/data/geoscan/raw/8603.pdf already exists
/nrcan_p2/data/geoscan/raw/8604.pdf already exists
/nrcan_p2/data/geoscan/raw/8605

/nrcan_p2/data/geoscan/raw/100326.zip already exists
/nrcan_p2/data/geoscan/raw/100327.pdf already exists
/nrcan_p2/data/geoscan/raw/100328.pdf already exists
/nrcan_p2/data/geoscan/raw/100329.zip already exists
/nrcan_p2/data/geoscan/raw/100330.zip already exists
/nrcan_p2/data/geoscan/raw/100331.pdf already exists
/nrcan_p2/data/geoscan/raw/100333.zip already exists
/nrcan_p2/data/geoscan/raw/100335.pdf already exists
/nrcan_p2/data/geoscan/raw/100337.pdf already exists
/nrcan_p2/data/geoscan/raw/100338.zip already exists
/nrcan_p2/data/geoscan/raw/100339.zip already exists
/nrcan_p2/data/geoscan/raw/100341.zip already exists
/nrcan_p2/data/geoscan/raw/100344.pdf already exists
/nrcan_p2/data/geoscan/raw/100345.zip already exists
/nrcan_p2/data/geoscan/raw/100349.zip already exists
/nrcan_p2/data/geoscan/raw/100350.zip already exists
/nrcan_p2/data/geoscan/raw/100351.pdf already exists
/nrcan_p2/data/geoscan/raw/100352.zip already exists
/nrcan_p2/data/geoscan/raw/100354.pdf already 

/nrcan_p2/data/geoscan/raw/100655.zip already exists
/nrcan_p2/data/geoscan/raw/100656.zip already exists
/nrcan_p2/data/geoscan/raw/100657.zip already exists
/nrcan_p2/data/geoscan/raw/100658.pdf already exists
/nrcan_p2/data/geoscan/raw/100659.zip already exists
/nrcan_p2/data/geoscan/raw/100660.pdf already exists
/nrcan_p2/data/geoscan/raw/100661.zip already exists
/nrcan_p2/data/geoscan/raw/100662.zip already exists
/nrcan_p2/data/geoscan/raw/100663.pdf already exists
/nrcan_p2/data/geoscan/raw/100664.zip already exists
/nrcan_p2/data/geoscan/raw/100665.zip already exists
/nrcan_p2/data/geoscan/raw/100666.pdf already exists
/nrcan_p2/data/geoscan/raw/100667.pdf already exists
/nrcan_p2/data/geoscan/raw/100668.zip already exists
/nrcan_p2/data/geoscan/raw/100669.zip already exists
/nrcan_p2/data/geoscan/raw/100670.pdf already exists
/nrcan_p2/data/geoscan/raw/100671.pdf already exists
/nrcan_p2/data/geoscan/raw/100673.pdf already exists
/nrcan_p2/data/geoscan/raw/100681.zip already 

/nrcan_p2/data/geoscan/raw/101008.zip already exists
/nrcan_p2/data/geoscan/raw/101009.zip already exists
/nrcan_p2/data/geoscan/raw/101010.pdf already exists
/nrcan_p2/data/geoscan/raw/101011.zip already exists
/nrcan_p2/data/geoscan/raw/101012.zip already exists
/nrcan_p2/data/geoscan/raw/101013.zip already exists
/nrcan_p2/data/geoscan/raw/101014.zip already exists
/nrcan_p2/data/geoscan/raw/101015.pdf already exists
/nrcan_p2/data/geoscan/raw/101016.zip already exists
/nrcan_p2/data/geoscan/raw/101017.pdf already exists
/nrcan_p2/data/geoscan/raw/101018.zip already exists
/nrcan_p2/data/geoscan/raw/101019.pdf already exists
/nrcan_p2/data/geoscan/raw/101020.pdf already exists
/nrcan_p2/data/geoscan/raw/101021.pdf already exists
/nrcan_p2/data/geoscan/raw/101022.zip already exists
/nrcan_p2/data/geoscan/raw/101023.zip already exists
/nrcan_p2/data/geoscan/raw/101024.pdf already exists
/nrcan_p2/data/geoscan/raw/101025.zip already exists
/nrcan_p2/data/geoscan/raw/101026.zip already 

...

/nrcan_p2/data/geoscan/raw/305396.zip already exists
/nrcan_p2/data/geoscan/raw/305397.zip already exists
/nrcan_p2/data/geoscan/raw/305398.pdf already exists
/nrcan_p2/data/geoscan/raw/305399.zip already exists
/nrcan_p2/data/geoscan/raw/305400.zip already exists
/nrcan_p2/data/geoscan/raw/305611.pdf already exists
/nrcan_p2/data/geoscan/raw/305827.zip already exists
/nrcan_p2/data/geoscan/raw/305828.zip already exists
/nrcan_p2/data/geoscan/raw/305829.pdf already exists
/nrcan_p2/data/geoscan/raw/305864.zip already exists
/nrcan_p2/data/geoscan/raw/305895.pdf already exists
/nrcan_p2/data/geoscan/raw/305896.pdf already exists
/nrcan_p2/data/geoscan/raw/305906.zip already exists
/nrcan_p2/data/geoscan/raw/305907.pdf already exists
/nrcan_p2/data/geoscan/raw/305912.zip already exists
/nrcan_p2/data/geoscan/raw/305956.pdf already exists
/nrcan_p2/data/geoscan/raw/305957.pdf already exists
/nrcan_p2/data/geoscan/raw/305958.pdf already exists
/nrcan_p2/data/geoscan/raw/305966.pdf already 

/nrcan_p2/data/geoscan/raw/308445.pdf already exists
/nrcan_p2/data/geoscan/raw/308450.zip already exists
/nrcan_p2/data/geoscan/raw/308454.zip already exists
/nrcan_p2/data/geoscan/raw/308463.zip already exists
/nrcan_p2/data/geoscan/raw/308482.pdf already exists
/nrcan_p2/data/geoscan/raw/308492.pdf already exists
/nrcan_p2/data/geoscan/raw/308495.pdf already exists
/nrcan_p2/data/geoscan/raw/308496.zip already exists
/nrcan_p2/data/geoscan/raw/311129.zip already exists
/nrcan_p2/data/geoscan/raw/311134.pdf already exists
/nrcan_p2/data/geoscan/raw/311135.pdf already exists
/nrcan_p2/data/geoscan/raw/311138.pdf already exists
/nrcan_p2/data/geoscan/raw/311141.pdf already exists
/nrcan_p2/data/geoscan/raw/311142.pdf already exists
/nrcan_p2/data/geoscan/raw/311143.pdf already exists
/nrcan_p2/data/geoscan/raw/311144.pdf already exists
/nrcan_p2/data/geoscan/raw/311145.pdf already exists
/nrcan_p2/data/geoscan/raw/311146.zip already exists
/nrcan_p2/data/geoscan/raw/311147.pdf already 

/nrcan_p2/data/geoscan/raw/313208.zip already exists
/nrcan_p2/data/geoscan/raw/313209.zip already exists
/nrcan_p2/data/geoscan/raw/313211.pdf already exists
/nrcan_p2/data/geoscan/raw/313212.pdf already exists
/nrcan_p2/data/geoscan/raw/313213.pdf already exists
/nrcan_p2/data/geoscan/raw/313214.pdf already exists
/nrcan_p2/data/geoscan/raw/313215.pdf already exists
/nrcan_p2/data/geoscan/raw/313216.pdf already exists
/nrcan_p2/data/geoscan/raw/313217.pdf already exists
/nrcan_p2/data/geoscan/raw/313218.pdf already exists
/nrcan_p2/data/geoscan/raw/313219.pdf already exists
/nrcan_p2/data/geoscan/raw/313220.pdf already exists
/nrcan_p2/data/geoscan/raw/313221.pdf already exists
/nrcan_p2/data/geoscan/raw/313222.pdf already exists
/nrcan_p2/data/geoscan/raw/313223.pdf already exists
/nrcan_p2/data/geoscan/raw/313224.pdf already exists
/nrcan_p2/data/geoscan/raw/313225.pdf already exists
/nrcan_p2/data/geoscan/raw/313226.pdf already exists
/nrcan_p2/data/geoscan/raw/313227.pdf already 

/nrcan_p2/data/geoscan/raw/314489.pdf already exists
/nrcan_p2/data/geoscan/raw/314490.pdf already exists
/nrcan_p2/data/geoscan/raw/314491.pdf already exists
/nrcan_p2/data/geoscan/raw/314495.pdf already exists
/nrcan_p2/data/geoscan/raw/314496.pdf already exists
/nrcan_p2/data/geoscan/raw/314497.pdf already exists
/nrcan_p2/data/geoscan/raw/314498.pdf already exists
/nrcan_p2/data/geoscan/raw/314499.pdf already exists
/nrcan_p2/data/geoscan/raw/314501.pdf already exists
/nrcan_p2/data/geoscan/raw/314502.pdf already exists
/nrcan_p2/data/geoscan/raw/314503.pdf already exists
/nrcan_p2/data/geoscan/raw/314506.pdf already exists
/nrcan_p2/data/geoscan/raw/314507.pdf already exists
/nrcan_p2/data/geoscan/raw/314508.pdf already exists
/nrcan_p2/data/geoscan/raw/314509.pdf already exists
/nrcan_p2/data/geoscan/raw/314510.pdf already exists
/nrcan_p2/data/geoscan/raw/314511.pdf already exists
/nrcan_p2/data/geoscan/raw/314512.pdf already exists
/nrcan_p2/data/geoscan/raw/314517.pdf already 

/nrcan_p2/data/geoscan/raw/315638.pdf already exists
/nrcan_p2/data/geoscan/raw/315639.pdf already exists
/nrcan_p2/data/geoscan/raw/315640.pdf already exists
/nrcan_p2/data/geoscan/raw/315641.pdf already exists
/nrcan_p2/data/geoscan/raw/315642.pdf already exists
/nrcan_p2/data/geoscan/raw/315643.pdf already exists
/nrcan_p2/data/geoscan/raw/315644.pdf already exists
/nrcan_p2/data/geoscan/raw/315645.pdf already exists
/nrcan_p2/data/geoscan/raw/315646.pdf already exists
/nrcan_p2/data/geoscan/raw/315647.zip already exists
/nrcan_p2/data/geoscan/raw/315661.zip already exists
/nrcan_p2/data/geoscan/raw/315662.pdf already exists
/nrcan_p2/data/geoscan/raw/315663.zip already exists
/nrcan_p2/data/geoscan/raw/315667.zip already exists
/nrcan_p2/data/geoscan/raw/315675.zip already exists
/nrcan_p2/data/geoscan/raw/315676.pdf already exists
/nrcan_p2/data/geoscan/raw/315684.pdf already exists
/nrcan_p2/data/geoscan/raw/315690.zip already exists
/nrcan_p2/data/geoscan/raw/315693.pdf already 

In [101]:
with open(path.join(geoscan_files_dir,"failed_to_download.csv"),"w") as f:
    f.write("geoscan_id,download_url,error\n")
    for err in failed_to_download:   
        f.write(f'{err[0][2]},{err[0][3]},{err[1]}\n')

In [7]:
print(f'Number of files failed to download: {len(failed_to_download)}')

NameError: name 'failed_to_download' is not defined

In [23]:
import os
sum(os.path.getsize(os.path.join(raw_dir,f)) for f in os.listdir(raw_dir))

735833379722

In [19]:
print(raw_dir)

/nrcan_p2/data/geoscan/raw


In [20]:
os.listdir(raw_dir)

['407.zip',
 '4680.pdf',
 '4681.pdf',
 '4682.pdf',
 '8273.pdf',
 '8274.pdf',
 '8275.pdf',
 '8276.pdf',
 '8277.pdf',
 '8278.pdf',
 '8279.pdf',
 '8280.pdf',
 '8281.pdf',
 '8282.pdf',
 '8283.pdf',
 '8284.pdf',
 '8285.pdf',
 '8286.pdf',
 '8287.pdf',
 '8288.zip',
 '8289.pdf',
 '8290.pdf',
 '8291.pdf',
 '8292.pdf',
 '8293.pdf',
 '8294.pdf',
 '8297.pdf',
 '8298.pdf',
 '8301.pdf',
 '8302.pdf',
 '8303.pdf',
 '8315.zip',
 '8318.pdf',
 '8320.zip',
 '8321.pdf',
 '8322.pdf',
 '8323.pdf',
 '8324.pdf',
 '8325.pdf',
 '8326.pdf',
 '8327.pdf',
 '8328.pdf',
 '8329.pdf',
 '8330.pdf',
 '8331.zip',
 '8332.zip',
 '8334.zip',
 '8335.zip',
 '8336.pdf',
 '8337.pdf',
 '8338.zip',
 '8339.pdf',
 '8347.pdf',
 '8348.pdf',
 '8350.pdf',
 '8351.pdf',
 '8355.pdf',
 '8356.zip',
 '8357.pdf',
 '8358.pdf',
 '8359.pdf',
 '8360.pdf',
 '8361.pdf',
 '8362.pdf',
 '8363.pdf',
 '8364.pdf',
 '8365.pdf',
 '8366.zip',
 '8367.pdf',
 '8368.pdf',
 '8369.pdf',
 '8372.pdf',
 '8373.pdf',
 '8374.pdf',
 '8375.pdf',
 '8376.pdf',
 '8377.pdf',


In [1]:
%%bash
mkdir -p /nrcan_p2/data/01_raw/20201006/geoscan/raw/pdf
mkdir -p /nrcan_p2/data/01_raw/20201006/geoscan/raw/zip
mv /nrcan_p2/data/01_raw/20201006/geoscan/raw/*.pdf /nrcan_p2/data/01_raw/20201006/geoscan/raw/pdf
mv /nrcan_p2/data/01_raw/20201006/geoscan/raw/*.zip /nrcan_p2/data/01_raw/20201006/geoscan/raw/zip

In [6]:
%%bash
ls -all /nrcan_p2/data

total 4
drwxr-xr-x 11 toolkit toolkit    0 Nov 10 20:03 .
drwxr-xr-x  1 root    root    4096 Nov  9 18:45 ..
drwxr-xr-x  5 toolkit toolkit    0 Nov 10 20:12 01_raw
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 02_intermediate
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 03_primary
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 04_feature
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 05_model_input
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 06_models
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 07_model_output
drwxr-xr-x  2 toolkit toolkit    0 Nov 10 19:29 08_reporting
drwxr-xr-x  2 toolkit toolkit    0 Nov  6 16:41 .ipynb_checkpoints
